# Using Model Card Toolkit

The Model Card Toolkit was archived on September 27, 2024. If you are still using this library, it’s time to explore alternatives. One option is to adopt the same schema and develop a custom documentation methodology tailored to your workflow. 

In this tutorial, we will demonstrate potential approaches and evaluate their feasibility for various organizations. 

This notebook includes code snippets from the TensorFlow example: [Standalone Model Card Toolkit Demo](https://www.tensorflow.org/responsible_ai/model_card_toolkit/examples/Standalone_Model_Card_Toolkit_Demo).

In [3]:
# https://pypi.org/project/model-card-toolkit/
# !pip install 'model-card-toolkit'

### Imports

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, BertTokenizer, AutoModel, BertForSequenceClassification, BertConfig, pipeline
import tempfile
import matplotlib.pyplot as plt
from IPython import display
import requests
import os
import zipfile

#import model_card_toolkit as mct
#from model_card_toolkit.documentation.examples import cats_vs_dogs
#from model_card_toolkit.utils.graphics import figure_to_base64str

## Model

**FinBERT** is a specialised language model built on the BERT architecture, designed specifically for tasks in the **financial domain**. It is fine-tuned on financial texts, such as earnings call transcripts, analyst reports, financial news, and other related datasets. The model is well-suited for sentiment analysis, named entity recognition, and text classification and question answering specific to finance.

Let's download the model from Huggingface.

In [2]:
model_name = 'yiyanghkust/finbert-tone'

finbert = BertForSequenceClassification.from_pretrained(model_name,num_labels=3, output_attentions=True)
#model = AutoModel.from_pretrained(model_name,num_labels=3, output_attentions=True)
model = AutoModel.from_pretrained(model_name,num_labels=3)
tokenizer = BertTokenizer.from_pretrained(model_name)
atokenizer = AutoTokenizer.from_pretrained(model_name)

config = BertConfig.from_pretrained(model_name)

/Users/asabuncuoglu/Documents/faid/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
from huggingface_hub import ModelCard
card = ModelCard.load(model_name)
print(card)

---
language: en
tags:
- financial-sentiment-analysis
- sentiment-analysis
widget:
- text: growth is strong and we have plenty of liquidity
---

`FinBERT` is a BERT model pre-trained on financial communication text. The purpose is to enhance financial NLP research and practice. It is trained on the following three financial communication corpus. The total corpora size is 4.9B tokens.
- Corporate Reports 10-K & 10-Q: 2.5B tokens
- Earnings Call Transcripts: 1.3B tokens
- Analyst Reports: 1.1B tokens

More technical details on `FinBERT`: [Click Link](https://github.com/yya518/FinBERT)

This released `finbert-tone` model is the `FinBERT` model fine-tuned on 10,000 manually annotated (positive, negative, neutral) sentences from analyst reports. This model achieves superior performance on financial tone analysis task. If you are simply interested in using `FinBERT` for financial tone analysis, give it a try.

If you use the model in your academic work, please cite the following paper:

Huan

## Updating Model Card for Reporting Fairness Evaluation

When usin the model card toolkit, the first step was to initialize a `ModelCardToolkit` object to maintain the [model card JSON file](https://github.com/tensorflow/model-card-toolkit/tree/master/model_card_toolkit/schema/) and [model card document](https://github.com/tensorflow/model-card-toolkit/tree/master/model_card_toolkit/template). `ModelCardToolkit.scaffold_assets()` were used to generate these assets and returned a `ModelCard` object.

```python
# https://github.com/tensorflow/model-card-toolkit/blob/master/model_card_toolkit/model_card_toolkit.py
model_card_dir = tempfile.mkdtemp()
toolkit = mct.ModelCardToolkit(model_card_dir)
# https://github.com/tensorflow/model-card-toolkit/blob/master/model_card_toolkit/model_card.py
model_card = toolkit.scaffold_assets()
```

You can continue using your existing model card JSON, since FAID uses the same schema for the model metadata.

In [5]:
import sys
sys.path.append('../../')
from faid import logging as faidlog

faidlog.init()

Model log file already exists
Data log file already exists
Risk log file already exists
Transparency log file already exists
Logging initialized


When you run the ```init()``` function, FAID generates four files: data.yml, model.yml, risks.yml, and transparency.yml.

```model.yml``` is a template document with empty values when it is first initialised.

### Annotate and Fill the Model Metadata

Similar to model card toolkit, FAID's `ModelCard` object has many fields that can be directly modified. These fields are rendered in the final generated Model documentation.


In [ ]:
model_card.model_details.name = 'Fine-tuned MobileNetV2 Model for Cats vs. Dogs'
model_card.model_details.overview = (
    'This model distinguishes cat and dog images. It uses the MobileNetV2 '
    'architecture (https://arxiv.org/abs/1801.04381) and is trained on the '
    'Cats vs Dogs dataset '
    '(https://www.tensorflow.org/datasets/catalog/cats_vs_dogs). This model '
    'performed with high accuracy on both Cat and Dog images.'
)
model_card.model_details.owners = [
  mct.Owner(name='Model Cards Team', contact='model-cards@google.com')
]
model_card.model_details.version = mct.Version(name='v1.0', date='08/28/2020')
model_card.model_details.references = [
    mct.Reference(reference='https://www.tensorflow.org/guide/keras/transfer_learning'),
    mct.Reference(reference='https://arxiv.org/abs/1801.04381'),
]
model_card.model_details.licenses = [mct.License(identifier='Apache-2.0')]
model_card.model_details.citations = [mct.Citation(citation='https://github.com/tensorflow/model-card-toolkit/blob/master/model_card_toolkit/documentation/examples/Standalone_Model_Card_Toolkit_Demo.ipynb')]

In [7]:
# Let's check metadata obtained from huggingface model
card.data

{'language': 'en', 'license': None, 'library_name': None, 'tags': ['financial-sentiment-analysis', 'sentiment-analysis'], 'base_model': None, 'datasets': None, 'metrics': None, 'eval_results': None, 'model_name': None, 'widget': [{'text': 'growth is strong and we have plenty of liquidity'}]}

In [8]:
# You can also access model card text from huggingface model
card.text

'\n`FinBERT` is a BERT model pre-trained on financial communication text. The purpose is to enhance financial NLP research and practice. It is trained on the following three financial communication corpus. The total corpora size is 4.9B tokens.\n- Corporate Reports 10-K & 10-Q: 2.5B tokens\n- Earnings Call Transcripts: 1.3B tokens\n- Analyst Reports: 1.1B tokens\n\nMore technical details on `FinBERT`: [Click Link](https://github.com/yya518/FinBERT)\n\nThis released `finbert-tone` model is the `FinBERT` model fine-tuned on 10,000 manually annotated (positive, negative, neutral) sentences from analyst reports. This model achieves superior performance on financial tone analysis task. If you are simply interested in using `FinBERT` for financial tone analysis, give it a try.\n\nIf you use the model in your academic work, please cite the following paper:\n\nHuang, Allen H., Hui Wang, and Yi Yang. "FinBERT: A Large Language Model for Extracting Information from Financial Text." *Contemporary

In [ ]:
# Start the log YAML file, you can see the output in the log folder in the root directory
faidlog.add_model_entry({
    "name": model_name,
    "description": card.text,
    "details": card.data.to_dict()
    })

Added model_info to model card


In [9]:
import re
import yaml
from collections import defaultdict

def parse_markdown_to_dict(markdown_content):
    """
    Parse Huggingface's Markdown content into a dictionary.
    :param markdown_content: str, the content of the markdown file.
    :return: dict, parsed data from the markdown.
    """
    result = defaultdict(str)
    
    # Regular expressions to capture headers and their content
    header_pattern = re.compile(r"^(#+)\s+(.*)")
    key_value_pattern = re.compile(r"^\*\*([^:]+):\*\*\s*(.*)")

    stack = []  # To keep track of the current hierarchy
    current_dict = result

    lines = markdown_content.splitlines()
    
    for line in lines:
        line = line.strip()
        
        if not line:
            continue

        header_match = header_pattern.match(line)
        key_value_match = key_value_pattern.match(line)

        if header_match:
            level = len(header_match.group(1))
            header = header_match.group(2).strip()
            
            # Adjust stack to the current header level
            while len(stack) >= level:
                stack.pop()
            
            stack.append(header)

            # Traverse the dictionary to the correct level
            current_dict = result
            for key in stack[:-1]:
                current_dict = current_dict[key]
            
            # Create a new sub-dictionary
            if header not in current_dict:
                current_dict[header] = {}
            current_dict = current_dict[header]
        
        elif key_value_match:
            key = key_value_match.group(1).strip()
            value = key_value_match.group(2).strip()
            current_dict[key] = value

    return result

parse_markdown_to_dict(card.text)

defaultdict(str, {'How to use': {}})

In [ ]:
# Alternatively you can use ModelCard class to get the model card
# If the model card is already available in the log directory, it will load the model card from the log directory
model_info = faidlog.ModelCard()
print(model_info.to_dict())

Model info is loaded from the model log file.
{'name': 'yiyanghkust/finbert-tone', 'description': '\n`FinBERT` is a BERT model pre-trained on financial communication text. The purpose is to enhance financial NLP research and practice. It is trained on the following three financial communication corpus. The total corpora size is 4.9B tokens.\n- Corporate Reports 10-K & 10-Q: 2.5B tokens\n- Earnings Call Transcripts: 1.3B tokens\n- Analyst Reports: 1.1B tokens\n\nMore technical details on `FinBERT`: [Click Link](https://github.com/yya518/FinBERT)\n\nThis released `finbert-tone` model is the `FinBERT` model fine-tuned on 10,000 manually annotated (positive, negative, neutral) sentences from analyst reports. This model achieves superior performance on financial tone analysis task. If you are simply interested in using `FinBERT` for financial tone analysis, give it a try.\n\nIf you use the model in your academic work, please cite the following paper:\n\nHuang, Allen H., Hui Wang, and Yi Yan

In [ ]:
model_info.set_model_detail("name", "BERT")
model_info.set_model_parameter("num_layers", 12)
model_info.add_quantitative_metric({"type": "accuracy", "value": 0.95})
model_info.add_consideration("ethical_considerations", "This model is not fair.")

In [ ]:
model_info.save()

Model info saved to the model log file.


In [ ]:
faidlog.get_model_entry()

{'model_info': {'name': 'yiyanghkust/finbert-tone',
  'description': '\n`FinBERT` is a BERT model pre-trained on financial communication text. The purpose is to enhance financial NLP research and practice. It is trained on the following three financial communication corpus. The total corpora size is 4.9B tokens.\n- Corporate Reports 10-K & 10-Q: 2.5B tokens\n- Earnings Call Transcripts: 1.3B tokens\n- Analyst Reports: 1.1B tokens\n\nMore technical details on `FinBERT`: [Click Link](https://github.com/yya518/FinBERT)\n\nThis released `finbert-tone` model is the `FinBERT` model fine-tuned on 10,000 manually annotated (positive, negative, neutral) sentences from analyst reports. This model achieves superior performance on financial tone analysis task. If you are simply interested in using `FinBERT` for financial tone analysis, give it a try.\n\nIf you use the model in your academic work, please cite the following paper:\n\nHuang, Allen H., Hui Wang, and Yi Yang. "FinBERT: A Large Language

In [ ]:
# When doesn't specify, the default key is "model_info"
# So, we can get the model card information by calling the following method
# faidlog.get_model_entry("model_info")
# Alternatively, you can get the model card information by calling the following method
faidlog.get_model_entry()

{'model_info': {'name': 'yiyanghkust/finbert-tone',
  'description': '\n`FinBERT` is a BERT model pre-trained on financial communication text. The purpose is to enhance financial NLP research and practice. It is trained on the following three financial communication corpus. The total corpora size is 4.9B tokens.\n- Corporate Reports 10-K & 10-Q: 2.5B tokens\n- Earnings Call Transcripts: 1.3B tokens\n- Analyst Reports: 1.1B tokens\n\nMore technical details on `FinBERT`: [Click Link](https://github.com/yya518/FinBERT)\n\nThis released `finbert-tone` model is the `FinBERT` model fine-tuned on 10,000 manually annotated (positive, negative, neutral) sentences from analyst reports. This model achieves superior performance on financial tone analysis task. If you are simply interested in using `FinBERT` for financial tone analysis, give it a try.\n\nIf you use the model in your academic work, please cite the following paper:\n\nHuang, Allen H., Hui Wang, and Yi Yang. "FinBERT: A Large Language

In [ ]:
faidlog.generate_model_card_report()
# The report will be saved in the reports/ folder in the root directory

##### Quantitative Analysis

`model_card.quantitative_analysis` contains information about a model's performance metrics.

Below, we create some synthetic performance metric values for a hypothetical model built on our dataset.

In [ ]:
model_card.quantitative_analysis.performance_metrics = [
  mct.PerformanceMetric(type='accuracy', value=str(accuracy)),
  mct.PerformanceMetric(type='accuracy', value=str(cat_accuracy), slice='cat'),
  mct.PerformanceMetric(type='accuracy', value=str(dog_accuracy), slice='Dog'),
]

##### Considerations

`model_card.considerations` contains qualifying information about your model - what are the appropriate use cases, what are limitations that users should keep in mind, what are the ethical considerations of application, etc.

In [ ]:
model_card.considerations.use_cases = [
    mct.UseCase(description='This model classifies images of cats and dogs.')
]
model_card.considerations.limitations = [
    mct.Limitation(description='This model is not able to classify images of other classes.')
]
model_card.considerations.ethical_considerations = [mct.Risk(
    name=
        'While distinguishing between cats and dogs is generally agreed to be '
        'a benign application of machine learning, harmful results can occur '
        'when the model attempts to classify images that don’t contain cats or '
        'dogs.',
    mitigation_strategy=
        'Avoid application on non-dog and non-cat images.'
)]

#### Graph Fields

It's often best practice for a report to provide information on a model's training data, and its performance across evaluation data. Model Card Toolkit allows users to encode this information in visualizations, rendered in the Model Card.

`model_card` has three sections for graphs -- `model_card.model_parameters.data.train.graphics` for training dataset statistics, `model_card.model_parameters.data.eval.graphics` for evaluation dataset statistics, and `model_card.quantitative_analysis.graphics` for quantitative analysis of model performance.

Graphs are stored as [base64 strings](https://en.wikipedia.org/wiki/Base64). If you have a [matplotlib](https://pypi.org/project/matplotlib/) figure, you can convert it to a base64 string with `model_card_toolkit.utils.graphics.figure_to_base64str()`.

In [ ]:
# Validation Set Size Bar Chart
fig, ax = plt.subplots()
width = 0.75
rects0 = ax.bar(0, len(examples['combined']['examples']), width, label='Overall')
rects1 = ax.bar(1, len(examples['cat']['examples']), width, label='Cat')
rects2 = ax.bar(2, len(examples['dog']['examples']), width, label='Dog')
ax.set_xticks(np.arange(3))
ax.set_xticklabels(['Overall', 'Cat', 'Dog'])
ax.set_ylabel('Validation Set Size')
ax.set_xlabel('Slices')
ax.set_title('Validation Set Size for Slices')
validation_set_size_barchart = figure_to_base64str(fig)

In [ ]:
# Acuracy Bar Chart
fig, ax = plt.subplots()
width = 0.75
rects0 = ax.bar(0, accuracy, width, label='Overall')
rects1 = ax.bar(1, cat_accuracy, width, label='Cat')
rects2 = ax.bar(2, dog_accuracy, width, label='Dog')
ax.set_xticks(np.arange(3))
ax.set_xticklabels(['Overall', 'Cat', 'Dog'])
ax.set_ylabel('Accuracy')
ax.set_xlabel('Slices')
ax.set_title('Accuracy on Slices')
accuracy_barchart = figure_to_base64str(fig)

Now we can add them to our `ModelCard`.

In [ ]:
model_card.model_parameters.data.append(mct.Dataset())
model_card.model_parameters.data[0].graphics.collection = [
  mct.Graphic(name='Validation Set Size', image=validation_set_size_barchart),
]
model_card.quantitative_analysis.graphics.collection = [
  mct.Graphic(name='Accuracy', image=accuracy_barchart),
]

### Generate the Model Card
Let's generate the Model Card document. Available formats are stored at [model_card_toolkit/template](https://github.com/tensorflow/model-card-toolkit/tree/master/model_card_toolkit/template). Here, we will demonstrate the HTML and Markdown formats.

First, we need to update the `ModelCardToolkit` with the latest `ModelCard`.

In [ ]:
toolkit.update_model_card(model_card)

Now, the `ModelCardToolkit` can generate a Model Card document with `ModelCardToolkit.export_format()`.

In [ ]:
# Generate a model card document in HTML (default)
html_doc = toolkit.export_format()

# Display the model card document in HTML
display.display(display.HTML(html_doc))

You can also output a Model Card in other formats, like Markdown.

In [ ]:
# Generate a model card document in Markdown
md_path = os.path.join(model_card_dir, 'template/md/default_template.md.jinja')
md_doc = toolkit.export_format(template_path=md_path, output_file='model_card.md')

# Display the model card document in Markdown
display.display(display.Markdown(md_doc))